In [1]:
# dataset
from google.cloud import bigquery
import json 

# maths and eda
import numpy as np
import pandas as pd
import itertools

# viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import warnings
warnings.filterwarnings("ignore")

# Load the credentials from the JSON file
with open('../../credentials.json', 'r') as f:
    creds_data = json.load(f)

import os
# Create the
#  credentials object
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../credentials.json"

# Initialize the BigQuery client
client = bigquery.Client(project="portfoliodata-428314")

import chart_studio
import chart_studio.plotly as py
f=open('../../plotly_credentials.json')
creds=json.load(f)

chart_studio.tools.set_credentials_file(username=creds['username'], api_key=creds['api_key'])

In [2]:
# query data
QUERY = ("""SELECT * FROM `portfoliodata-428314.kaggle.video_game_sales`""")
query_job = client.query(QUERY)
df = query_job.result().to_dataframe()
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Army Men 3D,PS,1999,Action,3DO,1.10,0.14,0.0,0.04,1.28,<NA>,<NA>,NaN,<NA>,None,None
1,Army Men: Air Attack,PS,1999,Action,3DO,0.47,0.32,0.0,0.06,0.85,<NA>,<NA>,NaN,<NA>,None,None
2,Army Men: Sarge's Heroes,N64,1999,Action,3DO,0.68,0.12,0.0,0.01,0.81,<NA>,<NA>,NaN,<NA>,None,None
3,Sammy Sosa High Heat Baseball 2001,PS,2000,Sports,3DO,0.32,0.22,0.0,0.04,0.58,<NA>,<NA>,NaN,<NA>,None,None
4,Army Men: Sarge's Heroes,PS,1999,Action,3DO,0.31,0.21,0.0,0.04,0.56,<NA>,<NA>,NaN,<NA>,None,None


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16719 non-null  object 
 3   Genre            16717 non-null  object 
 4   Publisher        16719 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   Int64  
 11  Critic_Count     8137 non-null   Int64  
 12  User_Score       7590 non-null   float64
 13  User_Count       7590 non-null   Int64  
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: Int64(3), float64(6), object(7)
memory usage: 2.1+ MB


In [4]:
df.describe()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count
count,16719.000000,16719.000000,16719.000000,16719.000000,16719.000000,8137.0,8137.0,7590.000000,7590.0
mean,0.263330,0.145025,0.077602,0.047332,0.533543,68.967679,26.360821,7.125046,162.229908
std,0.813514,0.503283,0.308818,0.186710,1.547935,13.938165,18.980495,1.500006,561.282326
min,0.000000,0.000000,0.000000,0.000000,0.010000,13.0,3.0,0.000000,4.0
25%,0.000000,0.000000,0.000000,0.000000,0.060000,60.0,12.0,6.400000,10.0
50%,0.080000,0.020000,0.000000,0.010000,0.170000,71.0,21.0,7.500000,24.0
75%,0.240000,0.110000,0.040000,0.030000,0.470000,79.0,36.0,8.200000,81.0
max,41.360000,28.960000,10.220000,10.570000,82.530000,98.0,113.0,9.700000,10665.0


In [5]:
len(df[['Name','Platform','Year_of_Release']].drop_duplicates())

16717

In [6]:
# drop duplicates and missing names
df=df.drop_duplicates(subset=['Name','Platform','Year_of_Release'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16717 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16716 non-null  object 
 1   Platform         16717 non-null  object 
 2   Year_of_Release  16717 non-null  object 
 3   Genre            16716 non-null  object 
 4   Publisher        16717 non-null  object 
 5   NA_Sales         16717 non-null  float64
 6   EU_Sales         16717 non-null  float64
 7   JP_Sales         16717 non-null  float64
 8   Other_Sales      16717 non-null  float64
 9   Global_Sales     16717 non-null  float64
 10  Critic_Score     8136 non-null   Int64  
 11  Critic_Count     8136 non-null   Int64  
 12  User_Score       7589 non-null   float64
 13  User_Count       7589 non-null   Int64  
 14  Developer        10095 non-null  object 
 15  Rating           9949 non-null   object 
dtypes: Int64(3), float64(6), object(7)
memory usage: 2.2+ MB


In [7]:
df[df['Name'].isna()]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
13677,None,GEN,1993,None,Acclaim Entertainment,1.78,0.53,0.0,0.08,2.39,<NA>,<NA>,NaN,<NA>,None,None


In [8]:
df=df[~df['Name'].isna()]

# EDA

## Platform

In [9]:
platform_df=df.Platform.value_counts(normalize=True,dropna=False).reset_index()
platform_df=platform_df[platform_df['proportion']>.03]
platform_df['proportion']=100*platform_df['proportion']
platform_df['pull']=0.0
platform_df.loc[0, 'pull'] = 0.2

In [10]:
import plotly.graph_objects as go
fig = px.pie(platform_df, values='proportion', names='Platform', title='Proportion of Video Games by Platform',
             hole=.4,color_discrete_sequence=px.colors.sequential.Purp_r,
            hover_data={},
            width=600)


fig.update_traces( texttemplate = "<b>%{label}</b> <br> %{value:.1f}%",
                   hovertemplate = "<b>%{label}</b> <br> Proportion of Total Video Games Released: %{value:.2f}%",
                 textinfo='value+label',
                  showlegend=False, textposition='inside',
                 pull=platform_df['pull'])
fig.show()

In [11]:
py.plot(fig, filename='video_game_platforms', auto_open=False)

'https://plotly.com/~vivianellis/1/'

In [12]:
Genre_df=df.Genre.value_counts(normalize=True,dropna=False).reset_index()
Genre_df=Genre_df[Genre_df['proportion']>.03]
Genre_df['proportion']=100*Genre_df['proportion']
Genre_df['pull']=0.0
Genre_df.loc[0, 'pull'] = 0.2

fig = px.pie(Genre_df, values='proportion', names='Genre', title='Genre of Video Games by Platform',
             hole=.4,color_discrete_sequence=px.colors.sequential.Purp_r,
            hover_data={},
            width=600)


fig.update_traces( texttemplate = "<b>%{label}</b> <br> %{value:.1f}%",
                   hovertemplate = "<b>%{label}</b> <br> Genre of Total Video Games Released: %{value:.2f}%",
                 textinfo='value+label',
                  showlegend=False, textposition='inside',
                 pull=Genre_df['pull'])
fig.show()

In [13]:
py.plot(fig, filename='video_game_platform_genres', auto_open=False)

'https://plotly.com/~vivianellis/3/'

## publishers by top global sales

In [14]:
top_sales=df[df['Global_Sales']>=df.Global_Sales.quantile(.9)]
top_4_publishers=top_sales.Publisher.value_counts(dropna=False).reset_index().head(4)
top_4_publishers

,Publisher,count
0,Nintendo,302
1,Electronic Arts,282
2,Activision,133
3,Sony Computer Entertainment,123


In [15]:
top_publisher_games=top_sales[top_sales['Publisher'].isin(top_4_publishers['Publisher'].unique())]
top_publisher_games=top_publisher_games[['Publisher','Genre','Name']].groupby(['Publisher','Genre']).count().reset_index()
top_publisher_games=top_publisher_games[top_publisher_games['Name']>9]
top_publisher_games=top_publisher_games.sort_values(by='Name',ascending=False).reset_index()
# rename sony for readablilty
top_publisher_games=top_publisher_games.replace({"Publisher": {'Sony Computer Entertainment':"Sony"}})

In [16]:
fig = px.sunburst(top_publisher_games, path=['Publisher', 'Genre'], values='Name', title='Publishers and Their Top Selling Genres',color_discrete_sequence=px.colors.sequential.Purpor_r)
fig.show()

In [17]:
py.plot(fig, filename='video_game_publishers_genres', auto_open=False)

'https://plotly.com/~vivianellis/5/'

## popularity of genre over the years

In [18]:
yearly_genre_sales=df[['Global_Sales','Year_of_Release','Genre']].groupby(['Year_of_Release','Genre']).sum().reset_index()
yearly_genre_sales=yearly_genre_sales[yearly_genre_sales['Year_of_Release']<='2016']
combinations = list(itertools.product(yearly_genre_sales.Year_of_Release.unique(), yearly_genre_sales.Genre.unique()))
year_genre=pd.DataFrame(data=combinations,columns=['Year_of_Release','Genre'])
yearly_genre_sales=yearly_genre_sales.merge(year_genre,how='outer')
top_genre_yearly_genre_sales=yearly_genre_sales.loc[yearly_genre_sales.groupby('Year_of_Release')['Global_Sales'].idxmax()]
top_genre_yearly_genre_sales['top']=1
yearly_genre_sales=yearly_genre_sales.merge(top_genre_yearly_genre_sales,how='outer')
yearly_genre_sales=yearly_genre_sales.fillna(0.0)
yearly_genre_sales['Year']=yearly_genre_sales['Year_of_Release'].astype(int)
yearly_genre_sales=yearly_genre_sales.sort_values(by=['Genre','Year'])
yearly_genre_sales

,Year_of_Release,Genre,Global_Sales,top,Year
0,1980,Action,0.34,0.0,1980
12,1981,Action,14.84,1.0,1981
24,1982,Action,6.52,0.0,1982
36,1983,Action,2.86,0.0,1983
48,1984,Action,1.85,0.0,1984
...,...,...,...,...,...
395,2012,Strategy,3.27,0.0,2012
407,2013,Strategy,6.09,0.0,2013
419,2014,Strategy,0.99,0.0,2014
431,2015,Strategy,1.84,0.0,2015


In [19]:
temp=df[['Name','Year_of_Release','Global_Sales','Genre','Publisher']].groupby(['Year_of_Release','Name','Genre','Publisher']).sum().reset_index()
temp=temp[temp['Year_of_Release']<='2016']
top_selling_game_yearly=temp.loc[temp.groupby('Year_of_Release')['Global_Sales'].idxmax()]
top_selling_game_yearly['Year']=top_selling_game_yearly['Year_of_Release'].astype(int)
top_selling_game_yearly = top_selling_game_yearly.rename(columns={'Name': 'Top Selling Game',
                                                                 'Genre':'Top Genre',
                                                                 'Publisher':'Top Publisher',
                                                                 'Global_Sales':'Top Global Sales'})

yearly_genre_sales=yearly_genre_sales.merge(top_selling_game_yearly,how='outer')

In [20]:
fig = px.bar(yearly_genre_sales, x="Genre", y="Global_Sales",color='top',animation_frame="Year",range_y=[0,105],template="simple_white",color_continuous_scale=px.colors.sequential.Purp,
            hover_data={"Genre":True,
                       "Global_Sales":True,
                       "Top Selling Game":True},
            custom_data=[yearly_genre_sales['Year'],yearly_genre_sales['Top Genre'], yearly_genre_sales['Top Selling Game'],yearly_genre_sales['Top Publisher'], yearly_genre_sales['Top Global Sales']],
            title='Annual Sales Trends in Video Game Genres')

fig.update_yaxes(showgrid=True, title_text='Global Sales in Millions')
fig.update_xaxes(title_text='')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1300
fig.layout.updatemenus[0].buttons[0].label='Play'
fig.layout.updatemenus[0].buttons[1].label='Pause'

fig.update_traces(hovertemplate="<b>%{x}</b><br><br>Total Global Sales in %{customdata[0]}: $%{y}M<br><br>Best Selling Game of The Year:<br> <b>%{customdata[2]}</b> <br> %{customdata[1]} - %{customdata[3]}, $%{customdata[4]}M")

fig.update_layout(coloraxis_showscale=False,
    legend_title_text='',
    font_family="Open Sans",
    font_size=16,
    title_font_family="Open Sans",
    title_font_size=24,
    title={
        'x': .5,
        'y': .97,
        'xanchor': 'center',
        'yanchor': 'top'}
)

for f in fig.frames:
    f.data[0].update(hovertemplate="<b>%{x}</b><br><br>Total Global Sales in %{customdata[0]}: $%{y}M<br><br>Best Selling Game of The Year:<br> <b>%{customdata[2]}</b> <br> %{customdata[1]} - %{customdata[3]}, $%{customdata[4]}M")

fig.show()

In [21]:
py.plot(fig, filename='video_game_annual_trends', auto_open=False)

'https://plotly.com/~vivianellis/7/'